In [1]:
%matplotlib inline
%load_ext autoreload

%autoreload 2

In [2]:
import json
import gff3_parsing
import pandas as pd
import glob

In [3]:
def analyze_genome(dataframe, energy_dict, gaps = (4,10), expected_len = 20):
    
    for index in dataframe.index:
        upstream = dataframe.loc[index,"upstream_sequence"]
        test_string = upstream.replace("T", "U")
        if len(test_string) != expected_len:
            continue
        if test_string.count("A") + test_string.count("U") +\
                                    test_string.count("C") + test_string.count("G") != expected_len:
            continue
            
        energy_list = []
        for gap in range(gaps[0],gaps[1]+1):
             energy_list.append(energy_dict[test_string[-gap - 6: -gap]])

        dataframe.at[index, "energy_binding"] = min(energy_list)
    return dataframe

# parameters 

In [3]:
sep = "\t"
upstream_len = 20

with open('../Data/energy_files/energyRef_CCUCCU_ensemble_noneConstraint.json', 'r') as infile:
       energy_dict = json.load(infile)

# Creating initial host tsv files

In [29]:
host_id = 717959
#host_id2 = 28450.2

host_df, host_genome = gff3_parsing.compile_sequences(["../Data/host_genomes/{}.gff3".format(host_id)],
                                                    ["../Data/host_genomes/{}.fasta".format(host_id)], upstream_len)
#combined_id = 28450
host_df = analyze_genome(host_df, energy_dict)
host_df.to_csv("../Data/host_genomes/{}.tsv".format(host_id), sep = sep )

# Creating initial viral tsv files

In [21]:
for gff_file in glob.glob("../Data/{}_rep_viruses/*.gff".format(host_id)):
    
   
    fasta_file = gff_file.replace(".gff", ".fasta")
    tsv_file = gff_file.replace(".gff", ".tsv")

    viral_df, viral_genome = gff3_parsing.compile_sequences([gff_file],[fasta_file], upstream_len)
    
    viral_df.to_csv(tsv_file, sep = sep )